# AI Project

### Group 6:
- AYOUB Simon
- GUG Maxime
- HOOG Cédric
- LANTZ Florian
- RINALDI Johan

## Context

## 1. Data handling & preparation

Our datasets are composed of 3 files:
- `employee_survey_data.csv`
- `general_data.csv`
- `manager_survey_data.csv`

Given that there is a common column in the 3 files, we will merge them into a single dataframe to facilitate the analysis.

In [70]:
import pandas as pd
import os

# Set the project root directory (Compatibility fix for Jupyter on linux)
pwd = !pwd
projectRoot = os.path.abspath(os.path.join(pwd[0], "..")) 

employee_survey_data = pd.read_csv(os.path.join(projectRoot, 'datasets', 'employee_survey_data.csv'))
general_data = pd.read_csv(os.path.join(projectRoot, 'datasets', 'general_data.csv'))
manager_survey_data = pd.read_csv(os.path.join(projectRoot, 'datasets', 'manager_survey_data.csv'))

data = pd.merge(employee_survey_data, general_data, on='EmployeeID')
data = pd.merge(data, manager_survey_data, on='EmployeeID')

data.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,...,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating
0,1,3.0,4.0,2.0,51,No,Travel_Rarely,Sales,6,2,...,11,8,0,1.0,6,1,0,0,3,3
1,2,3.0,2.0,4.0,31,Yes,Travel_Frequently,Research & Development,10,1,...,23,8,1,6.0,3,5,1,4,2,4
2,3,2.0,2.0,1.0,32,No,Travel_Frequently,Research & Development,17,4,...,15,8,3,5.0,2,5,0,3,3,3
3,4,4.0,4.0,3.0,38,No,Non-Travel,Research & Development,2,5,...,11,8,3,13.0,5,8,7,5,2,3
4,5,4.0,1.0,3.0,32,No,Travel_Rarely,Research & Development,10,1,...,12,8,2,9.0,2,6,0,4,3,3


Now that we have merged the datasets, we can start the data preparation.

### 1.1 Data cleaning
First, we will check if there are any missing values in the dataset.

In [71]:
data.isnull().sum()

EmployeeID                  0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
JobInvolvement              0
PerformanceRating           0
dtype: int64

We can see that 3 columns have missing values:
- `EnvironmentSatisfaction`
- `JobSatisfaction`
- `WorkLifeBalance`

These values all come from the employee survey data, so we can assume that some employees didn't answer the survey.

Given that this data is numerical, we will fill the missing values with the median of the column.

In [72]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

data[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']] = imputer.fit_transform(data[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']])

data.isnull().sum()

EmployeeID                  0
EnvironmentSatisfaction     0
JobSatisfaction             0
WorkLifeBalance             0
Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
JobInvolvement              0
PerformanceRating           0
dtype: int64

Now that we have filled the missing values, we can move on to the next step.

### 1.2 Data transformation
In this step, we will transform the categorical data into numerical data. This will allow us to use the data in machine learning models.

First, we need to list the categorical columns.

In [73]:
data.select_dtypes(include=['object']).columns

Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender',
       'JobRole', 'MaritalStatus', 'Over18'],
      dtype='object')

Then we'll analyze the unique values of each column to see if we can transform them into numerical data.

In [74]:
for column in data.select_dtypes(include=['object']).columns:
    print(column, data[column].unique())

Attrition ['No' 'Yes']
BusinessTravel ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Department ['Sales' 'Research & Development' 'Human Resources']
EducationField ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
Gender ['Female' 'Male']
JobRole ['Healthcare Representative' 'Research Scientist' 'Sales Executive'
 'Human Resources' 'Research Director' 'Laboratory Technician'
 'Manufacturing Director' 'Sales Representative' 'Manager']
MaritalStatus ['Married' 'Single' 'Divorced']
Over18 ['Y']


We can see that the following columns can be transformed into numerical data:
- `Attrition`
- `BusinessTravel`
- `Department`
- `EducationField`
- `Gender`
- `JobRole`
- `MaritalStatus`
- `Over18`

We will use `sklearn` encoders to transform the data.
Given that we have both data for which order is important and data for which order is not important, we will use both `OrdinalEncoder` and `OneHotEncoder`. We will use `OrdinalEncoder` for the data for which order is important, therefore associating a numerical value to each unique value. We will use `OneHotEncoder` for the data for which order is not important, therefore creating new binary columns for each unique value.

The following columns will be transformed using `OrdinalEncoder`:
- `Attrition`
- `BusinessTravel`
- `Gender`
- `MaritalStatus`

The following columns will be transformed using `OneHotEncoder`:
- `Department`
- `EducationField`
- `JobRole`
- `Over18`

In [75]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Create the encoders
ordinal_encoder = OrdinalEncoder()
onehot_encoder = OneHotEncoder(sparse_output=False) 

# Transform the ordinal data 
ordinal_columns = ['Attrition', 'BusinessTravel', 'Gender', 'MaritalStatus']
data[ordinal_columns] = ordinal_encoder.fit_transform(data[ordinal_columns])

# Transform the onehot data
onehot_columns = ['Department', 'EducationField', 'JobRole', 'Over18']
onehot_data = onehot_encoder.fit_transform(data[onehot_columns])
onehot_data = pd.DataFrame(onehot_data, columns=onehot_encoder.get_feature_names_out(onehot_columns))
data = pd.concat([data, onehot_data], axis=1)
data = data.drop(columns=onehot_columns)

data.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,DistanceFromHome,Education,EmployeeCount,...,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,Over18_Y
0,1,3.0,4.0,2.0,51,0.0,2.0,6,2,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,3.0,2.0,4.0,31,1.0,1.0,10,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,3,2.0,2.0,1.0,32,0.0,1.0,17,4,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,4,4.0,4.0,3.0,38,0.0,0.0,2,5,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,4.0,1.0,3.0,32,0.0,2.0,10,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


We have now transformed the categorical data into numerical data. We can now move on to the next step.

### 1.3 Removing irrelevant columns

In this step, we will remove the columns that are not relevant to the analysis.
We will remove the following columns because their value is the same throughout the dataset:
- `EmployeeCount`
- `Over18`
- `StandardHours`

We will also remove the `EmployeeID` column because it is not relevant to the analysis either.

In [76]:
data = data.drop(columns=['EmployeeCount', 'Over18_Y', 'StandardHours', 'EmployeeID'])

### 1.4 Ethical considerations

Given that we have to take a decision, we need to consider the ethical implications of certain attributes.

We have chosen not to consider the following columns in our analysis because they are not ethical:
- `Gender`
- `MaritalStatus`

We will therefore remove these columns from the dataset, not because they are irrelevant, but because we strive to be ethical in our analysis.



In [77]:
# Remove the columns that are not ethical
data = data.drop(columns=['Age', 'MaritalStatus'])

data.head()

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Attrition,BusinessTravel,DistanceFromHome,Education,Gender,JobLevel,MonthlyIncome,...,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,3.0,4.0,2.0,0.0,2.0,6,2,0.0,1,131160,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,2.0,4.0,1.0,1.0,10,1,0.0,1,41890,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2.0,2.0,1.0,0.0,1.0,17,4,1.0,4,193280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4.0,4.0,3.0,0.0,0.0,2,5,1.0,3,83210,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,1.0,3.0,0.0,2.0,10,1,1.0,1,23420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
